In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import rioxarray as rio

import pandas as pd
from pathlib import Path
from pystac_client import Client
import folium

from src.utils import SentinelDownloader

root = Path.cwd()

In [2]:
p = gpd.read_file(root / 'suny' / 'tree_points_2016_2013.gpkg')
p = p.to_crs(26918)

p1 = p.loc[p.YEAR==2016]

In [ ]:
sn_2019 = SentinelDownloader(2019,p1,26918,drop='last')
sn_2019.plot_initial_data()   # use october from 2019, rest from 2018

# get two october images from 2019 data
sn_2019_oct = sn_2019.cloud_masked_and_scaled.isel(time=[34,36])

In [ ]:
sn = SentinelDownloader(2018,p1,26918,drop='last')
sn.plot_initial_data()

# get most complete images from 2018 data
sn_select = sn.cloud_masked_and_scaled.isel(time=[7,10,12,19,20,22,30,32,35,36,38])

number of low cloud images: 117


In [ ]:
# adjust time coordinates from 2019 to 2018
sn_2019_oct['time'] = sn_2019_oct.time.to_index() - pd.Timedelta(days=365)
# add october data to 2018 time series
combine = sn_select.combine_first(sn_2019_oct)
# set complete data attribute
sn.complete_data = combine

sn.save_bands_data(root=root,filename='2018_suny_dmf')

In [23]:
sn.get_indices()
sn.save_vi_data(root=root,filename='2018_suny_dmf')

In [27]:
sn.check_data_range(sn.complete_data)

Band 0: min: 0.0001, max 1.3464
Band 1: min: 0.0001, max 1.344
Band 2: min: 0.0003, max 1.3584
Band 3: min: 0.0001, max 1.2766
Band 4: min: 0.0001, max 1.2085
Band 5: min: 0.0001, max 1.1759
Band 6: min: 0.0001, max 1.3112
Band 7: min: 0.0001, max 1.1289
Band 8: min: 0.0031, max 0.6571
Band 9: min: 0.0001, max 0.5714
